In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from astropy.coordinates import SkyCoord
import os
import sys
sys.path.insert(0, '/home/aew492/lss-dipoles')
import tools
from Secrest_dipole import SecrestDipole
from multipoles import multipole_map
import dipole

In [ ]:
def fit_multipole(map_to_fit, template_maps, Cinv=None, fit_zeros=False, idx=None):
    """
    Fits multipole amplitudes to an input healpix density map.
    
    Parameters
    ----------
    map_to_fit : 1D array-like, length npix
        Input healpix map.
    template_maps : 2D array-like, shape (n,npix)
        Y_lm templates to fit.
    Cinv : array-like, optional
        Inverse covariance matrix. If 1D, taken to be the diagonal terms.
    fit_zeros : bool, optional
        Whether to fit zero-valued pixels in `map_to_fit`. The default is False.
    idx : array-like, optional
        Pixel indices to fit.
    
    Returns
    -------
    bestfit_pars :
        The 2 * ell + 1 best-fit amplitudes corresponding to each template map.
    bestfit_stderr :
        The standard error on the fit.
    
    """
    assert map_to_fit.ndim == 1, "input map must be 1-dimensional"
    assert len(map_to_fit) == template_maps.shape[1], "input map and template maps must have the same NPIX"
    
    NPIX = len(map_to_fit)
    # design matrix
    A = template_maps.T
    # covariances: identity for now
    if Cinv is None:
        Cinv = np.ones(NPIX)
    else:
        assert len(Cinv) == NPIX, "input Cinv and input map must have the same length"

    # indices to fit
    idx_to_fit = np.full(NPIX, True)
    if fit_zeros is False:
        idx_to_fit = idx_to_fit & (map_to_fit!=0.)
    if idx is not None:
        assert len(idx) == NPIX, "input idx and input map must have the same length"
        idx_to_fit = idx_to_fit & idx
    map_to_fit, A, Cinv = map_to_fit[idx_to_fit], A[idx_to_fit], Cinv[idx_to_fit]

    # perform the regression
    bestfit_pars, bestfit_Cinv = tools.lstsq(map_to_fit, A, Cinv)

    # uncertainties on the best-fit pars
    bestfit_stderr = np.sqrt(np.diag(np.linalg.inv(bestfit_Cinv)))

    return bestfit_pars, bestfit_stderr

In [ ]:
def construct_templates(ells, NSIDE=64):
    """
    Returns a (n,npix) array of Y_lm templates; the design matrix used to fit multipoles to a healpix map.
    
    Parameters
    ----------
    ells : int or array-like
        The degrees to construct.
    NSIDE : int, optional
        The healpix resolution.
    
    Returns
    -------
    templatess : (n,npix) array
        The design matrix: each column corresponds to a Ylm template. n is 2ell+1 summed over the input ells.
        
    """
    # check/adjust input ells
    ells = np.array(ells).astype(int)
    assert ells.ndim <= 1
    # if input is a single value
    if ells.ndim == 0:
        ells = ells[...,np.newaxis]
    
    # construct templates for each ell and append to 
    n = np.sum([2 * ell + 1 for ell in ells])
    templatess = np.empty((n,hp.nside2npix(NSIDE)))
    it = 0  # keep track of the column index
    for ell in ells:
        templates = np.array([
            multipole_map(amps, NSIDE=NSIDE) for amps in np.identity(2 * ell + 1)
        ])
        templatess[it:it + 2 * ell + 1] = templates
        it += 2 * ell + 1
    
    return templatess

### inputs

In [ ]:
# inputs used across the entire notebook
NSIDE = 64

# kwargs for each sample to pass to SecrestDipole() to load
catwise_kwargs = dict(initial_catfn='catwise_agns_master.fits', catname='catwise_agns', mag='w1',
                      blim=30, maglim=16.4, load_init=False)
quaia_kwargs = dict(initial_catfn='quaia_G20.0.fits', catname='quaia', mag='G',
                    blim=30, maglim=20., save_tag='_r1.0', load_init=False, compcorrect=True)

### load sample

In [ ]:
# This notebook should check for the files and download them from the web if they aren't here

In [ ]:
# load the source density table for the final sample (masked and density-corrected)
d = SecrestDipole(**catwise_kwargs)
map_ = d.load_hpxelatcorr()

In [ ]:
# construct map from source density table
map_to_fit = np.empty(hp.nside2npix(NSIDE))
map_to_fit[:] = np.nan
map_to_fit[map_['hpidx']] = map_['elatdenscorr']
mean, std = np.nanmean(map_to_fit), np.nanstd(map_to_fit)
fig = plt.figure(figsize=(8,4))
hp.mollview(map_to_fit, coord=['C','G'], title=f'Input map: {d.catname}', unit='sources per healpixel',
            badcolor='w', min=mean-2*std, max=mean+2*std, fig=fig)
hp.graticule()

### monopole

In [ ]:
# construct the monopole template
monopole_template = construct_templates(0, NSIDE=NSIDE)

### dipole

In [ ]:
# construct dipole templates
dipole_templates = construct_templates(1, NSIDE=NSIDE)

# plot
fig = plt.figure(figsize=(12,2))
titles = ['m = -1', 'm = 0', 'm = 1']
for i, template in enumerate(dipole_templates):
    hp.mollview(template, coord=['C','G'], title=titles[i],
                sub=(1,len(titles),i+1), min=-.6, max=.6, cmap='coolwarm', fig=fig)
fig.suptitle('Dipole templates', y=1.2)

In [ ]:
# fit a monopole + dipole to the density map we loaded above
templates = np.concatenate([monopole_template, dipole_templates])
pars, stderr = fit_multipole(map_to_fit, templates, idx=~np.isnan(map_to_fit))
print("best-fit pars: ", pars)

# plot the best-fit map
bestfit_dipmap = pars[1:] @ templates[1:] / (pars[0] * templates[0])
fig = plt.figure(figsize=(7,3))
hp.mollview(bestfit_dipmap, coord=['C','G'], title='Recovered dimensionless dipole', fig=fig)

In [ ]:
# the best-fit monopole amplitude times Y_00 gives us the mean source density in the map
0.5 * np.sqrt(1/np.pi) * pars[0]

In [ ]:
# does this recovered amplitude and direction match the old way of doing the fit?
amp = np.linalg.norm(pars[1:]/pars[0])
# manually tell healpy which parameters correspond to the x, y, and z directions
direction = hp.vec2dir(pars[3], vy=pars[1], vz=pars[2])
direction = SkyCoord(direction[1], np.pi/2 - direction[0], frame='icrs', unit='rad')
amp, direction.galactic

In [ ]:
# The direction is the correct fiducial, but note that the amplitude definition no longer holds...

### quadrupole

In [ ]:
# construct quadrupole templates
quadrupole_templates = construct_templates(2, NSIDE=NSIDE)

# plot
fig = plt.figure(figsize=(12,1.5))
titles = ['m = -2', 'm = -1', 'm = 0', 'm = 1', 'm = 2']
for i, template in enumerate(quadrupole_templates):
    hp.mollview(template, coord=['C','G'], title=titles[i],
                sub=(1,len(titles),i+1), min=-.6, max=.6, cmap='coolwarm', fig=fig)
fig.suptitle('Quadrupole templates', y=1.2)

In [ ]:
# fit a monopole + quadrupole to the density map we loaded above
templates = np.concatenate([monopole_template, quadrupole_templates])
pars, stderr = fit_multipole(map_to_fit, templates, idx=~np.isnan(map_to_fit))
print("best-fit pars: ", pars)

# plot the best-fit map scaled by the monopole
bestfit_quadmap = pars[1:] @ templates[1:] / (pars[0] * templates[0])
fig = plt.figure(figsize=(7,3))
hp.mollview(bestfit_quadmap, coord=['C','G'], title='Recovered dimensionless quadrupole', fig=fig)

In [ ]:
# what is the dimensionless amplitude?
amp = np.linalg.norm(pars[1:]/pars[0])
amp

### octupole

In [ ]:
# construct octupole templates
octupole_templates = construct_templates(3, NSIDE=NSIDE)

# plot
fig = plt.figure(figsize=(13,1.2))
titles = ['m = -3', 'm = -2', 'm = -1', 'm = 0', 'm = 1', 'm = 2', 'm = 3']
for i, template in enumerate(octupole_templates):
    hp.mollview(template, coord=['C','G'], title=titles[i],
                sub=(1,len(titles),i+1), min=-.6, max=.6, cmap='coolwarm', fig=fig)
fig.suptitle('Octupole templates', y=1.2)

In [ ]:
# fit a monopole + quadrupole to the density map we loaded above
templates = np.concatenate([monopole_template, octupole_templates])
pars, stderr = fit_multipole(map_to_fit, templates, idx=~np.isnan(map_to_fit))
print("best-fit pars: ", pars)

# plot the best-fit map scaled by the monopole
bestfit_octmap = pars[1:] @ templates[1:] / (pars[0] * templates[0])
fig = plt.figure(figsize=(7,3))
hp.mollview(bestfit_octmap, coord=['C','G'], title='Recovered dimensionless octupole', fig=fig)

In [ ]:
amp = np.linalg.norm(pars[1:]/pars[0])
amp

### dipole + quadrupole

In [ ]:
# simultaneously fit a monopole + dipole + quadrupole to the density map we loaded above
templates = np.concatenate([monopole_template, dipole_templates, quadrupole_templates])
pars, stderr = fit_multipole(map_to_fit, templates, idx=~np.isnan(map_to_fit))
print("best-fit pars: ", pars)

# plot the best-fit map
bestfit_map = pars @ templates
fig = plt.figure(figsize=(7,3))
hp.mollview(bestfit_map, coord=['C','G'], title=r'Recovered $\ell=0,1,2$', fig=fig)

### dipole + quadrupole + octupole

In [ ]:
# simultaneously fit a monopole + dipole + quadrupole + octupole to the density map we loaded above
templates = np.concatenate([monopole_template, dipole_templates, quadrupole_templates, octupole_templates])

fig = plt.figure(figsize=(10,2))
# plot the templates 
for i, template in enumerate(templates):
    hp.mollview(template, coord=['C','G'], title='', sub=(2,round(len(templates)//2),i+1),
                min=-.6, max=.6, cmap='coolwarm', fig=fig)
fig.suptitle(r'$\ell=0,1,2,3$ templates')

# perform the fit
pars, stderr = fit_multipole(map_to_fit, templates, idx=~np.isnan(map_to_fit))
print("best-fit pars: ", pars)

# plot the best-fit map
bestfit_map = pars @ templates
fig = plt.figure(figsize=(7,3))
hp.mollview(bestfit_map, coord=['C','G'], title=r'Recovered $\ell=0,1,2,3$', fig=fig)

### $\hat{C}_\ell$

Remember that any well-behaved function of $\theta$ and $\phi$ can be expressed entirely in terms of spherical harmonics (completeness property):
$$
f(\theta,\phi) = \sum_{\ell=0}^{\infty}\sum_{m=-\ell}^{\ell} a_{\ell m}\,Y_{\ell m}
$$

Define our estimate as
$$
\hat{C}_\ell = \frac{1}{2\ell +1}\,\sum_{m=-\ell}^{\ell} | a_{\ell m} |^2
$$

In [ ]:
def compute_Cells(amps):
    """
    Returns the power C(ell) for several ells given a list of amplitudes corresponding to the a_lm coefficients
    for each ell, increasing from ell=0.
    """
    ell = 0
    i1 = 0
    Cells = np.array([])
    while i1 < len(amps):
        i2 = i1 + 2 * ell + 1
        assert i2 <= len(amps)
        Cell = compute_Cell(amps[i1:i2])
        Cells = np.append(Cells, Cell)
        ell += 1
        i1 = i2
    return Cells

In [ ]:
def compute_Cell(alms):
    """
    Returns the power C(ell) given a list of coefficients a_lm for a single ell.
    """
    assert alms.ndim <= 1
    # pad if aellems is a scalar:
    if alms.ndim == 0:
        alms = alms[..., np.newaxis]
    # infer ell from the number of moments 2ell+1
    ell = (len(alms) - 1) // 2
    assert np.mean(alms**2) == np.sum(alms**2)/(2*ell+1)
    return np.mean(alms**2)

In [ ]:
# simultaneously fit all low-ell amplitudes
ells = np.arange(0, 8)
templates = construct_templates(ells, NSIDE=NSIDE)
    
pars, stderr = fit_multipole(map_to_fit, templates, idx=~np.isnan(map_to_fit))

In [ ]:
# compute Cells, divide by monopole to make dimensionless
Cells = compute_Cells(pars/pars[0])

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(ells[1:], Cells[1:], 'ks')
ax.axhline(0, c='k', lw=0.5, alpha=0.5)
ax.grid(lw=0.5, alpha=0.5)
ax.set_ylim((-4e-6,None))
ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$\hat{C}_\ell\,/\hat{C}_0$')
ax.set_title(f'{d.catname}'r' low-$\ell$ power spectrum')

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(ells[1:], ells[1:] * (ells[1:] + 1) * Cells[1:], 'ks')
ax.axhline(0, c='k', lw=0.5, alpha=0.5)
ax.grid(lw=0.5, alpha=0.5)
ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$\ell\,(\ell +1)\,\hat{C}_\ell\,/\hat{C}_0$')
ax.set_title(f'{d.catname}'r' low-$\ell$ power spectrum')

In [ ]:
# fit only monopole and dipole
# simultaneously fit all low-ell amplitudes
templates = construct_templates([0,1], NSIDE=NSIDE)
    
pars, stderr = fit_multipole(map_to_fit, templates, idx=~np.isnan(map_to_fit))
pars

In [ ]:
compute_Cell(pars[1:])/pars[0]

In [ ]:
# check if the expected combination of new pars/alms and prefactors matches the dipole amplitude that we get
#   using the old formula and old pars: 0.015

# prefactors
A = 0.5 * np.sqrt(1/np.pi)
B = 0.5 * np.sqrt(3/(2*np.pi))
C = 0.5 * np.sqrt(3/np.pi)
D = -0.5 * np.sqrt(3/(2*np.pi))

dipamp = np.linalg.norm(np.array([B*pars[1], C*pars[2], D*pars[3]])) / (A*pars[0])
dipamp

In [ ]:
A * pars[0]

In [ ]:
np.nanmean(map_to_fit)